<a href="https://colab.research.google.com/github/sharmayash1824/Import_data_cleaning/blob/main/Import_data_cleaning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [20]:
import pandas as pd
import re

def clean_product_descriptions(file_path, sheet_name=0, column_name='PRODUCT DESCRIPTION'):
    # Load the Excel file
    df = pd.read_excel(file_path, sheet_name=sheet_name)

    # Ensure the column exists
    if column_name not in df.columns:
        raise ValueError(f"Column '{column_name}' not found in the Excel file.")

    # Remove leading/trailing whitespace
    df[column_name] = df[column_name].astype(str).str.strip()

    # Function to remove repetitive phrases in product descriptions while ensuring correct formatting
    def remove_redundancy(desc):
        words = desc.split()
        for i in range(1, len(words)):
            phrase = " ".join(words[:i])
            if desc.count(phrase) > 1:
                cleaned_desc = desc[:desc.rfind(phrase)].strip()
                # Ensure meaningful data is retained
                if len(cleaned_desc) > len(desc) * 0.6:
                    return cleaned_desc
        return desc  # Keep original if no significant repetition found

    df[column_name] = df[column_name].apply(remove_redundancy)

    # Save cleaned data to a new file without dropping duplicate rows
    output_file = file_path.replace('.xlsx', '_cleaned.xlsx')
    df.to_excel(output_file, index=False)

    print(f"Cleaned data saved to: {output_file}")
    return df

# Example usage
file_path = "Import_Data Analytics.xlsx"  # Update this path if needed
cleaned_df = clean_product_descriptions(file_path)


Cleaned data saved to: Import_Data Analytics_cleaned.xlsx


Remove repeat phrases.


In [21]:

import pandas as pd
import re

def clean_product_descriptions(file_path, sheet_name=0, column_name='PRODUCT DESCRIPTION'):
    # Load the Excel file
    df = pd.read_excel(file_path, sheet_name=sheet_name)

    # Ensure the column exists
    if column_name not in df.columns:
        raise ValueError(f"Column '{column_name}' not found in the Excel file.")

    # Remove leading/trailing whitespace
    df[column_name] = df[column_name].astype(str).str.strip()

    # Function to remove repetitive phrases in product descriptions while ensuring correct formatting
    def remove_redundancy(desc):
        match = re.search(r'^(.*?)(?: \1)+$', desc)
        if match:
            return match.group(1).strip()
        return desc  # Keep original if no significant repetition found

    df[column_name] = df[column_name].apply(remove_redundancy)

    # Save cleaned data to a new file without dropping duplicate rows
    output_file = file_path.replace('.xlsx', '_cleaned.xlsx')
    df.to_excel(output_file, index=False)

    print(f"Cleaned data saved to: {output_file}")
    return df

# Example usage
file_path = "Import_Data Analytics_cleaned.xlsx"  # Update this path if needed
cleaned_df = clean_product_descriptions(file_path)


Cleaned data saved to: Import_Data Analytics_cleaned_cleaned.xlsx


Add spaces before and after parenthesis.(but not in case of "-("  beacause we have some grades in this
 format
)
:

In [22]:
import pandas as pd
import re

# Function to ensure proper spacing around parentheses
def fix_parentheses_spacing(text):
    # Case 1: If '(' is preceded by '-', only add space after '('
    text = re.sub(r'-(\()', r'- ( ', text)

    # Case 2: Otherwise, ensure spaces before and after '('
    text = re.sub(r'(?<!-)(\S)\(', r'\1 ( ', text)

    # Ensure space before and after ')'
    text = re.sub(r'\)(\S)', r' ) \1', text)

    return text

# Load data
file_path = "Import_Data Analytics_cleaned_cleaned.xlsx"  # Change to your actual file path
df = pd.read_excel(file_path)  # Use pd.read_csv(file_path) for CSV

# Apply transformation to "PRODUCT DESCRIPTION"
df["PRODUCT DESCRIPTION"] = df["PRODUCT DESCRIPTION"].astype(str).apply(fix_parentheses_spacing)

# Save the modified file
output_path = "cleaned_file.xlsx"
df.to_excel(output_path, index=False)  # Use df.to_csv("cleaned_file.csv", index=False) for CSV

print(f"File saved as {output_path}")

File saved as cleaned_file.xlsx


Add space after word GRADE if not followed by "-" and if it is followed by "-" add space after hyphen


In [23]:

import pandas as pd
import re

# Load Excel file
file_path = "cleaned_file.xlsx"  # Change this to your file path
df = pd.read_excel(file_path)

# Function to modify PRODUCT DESCRIPTION
def modify_description(description):
    if pd.isna(description):  # Handle NaN values
        return description

    # List of words to modify
    words_to_modify = ["grade", "thickness","width","finish","quality"]

    for word in words_to_modify:
        # Add space after 'word' if not followed by space, '-', or ':'
        description = re.sub(rf'(?i)\b({word})(?=[A-Za-z0-9:])', r'\1 ', description)
        # Ensure space after ':' or '-' only if they are just after 'word' without space
        description = re.sub(rf'(?i)\b({word}):', r'\1: ', description)
        description = re.sub(rf'(?i)\b({word})-', r'\1- ', description)

    # Ensure space after every ':'
    description = re.sub(r':(?=[^\s])', r': ', description)

    return description

# Apply function to PRODUCT DESCRIPTION column
df["PRODUCT DESCRIPTION"] = df["PRODUCT DESCRIPTION"].astype(str).apply(modify_description)

# Save modified file
output_file = "modified_file.xlsx"
df.to_excel(output_file, index=False)
print(f"Modified file saved as {output_file}")


Modified file saved as modified_file.xlsx


Extracting Grade,Finish and Quality

In [26]:
# import pandas as pd
# import json
# import re

# # Load Excel file
# file_path = "modified_file.xlsx"  # Update with your actual file path
# df = pd.read_excel(file_path)

# # Load grade and finish lists from JSON
# with open("Grades_json.json", "r") as f:
#     grade_list = json.load(f)  # List of possible grades

# with open("Finish_json.json", "r") as f:
#     finish_list = json.load(f)  # List of possible finishes

# # Function to extract all occurrences of grade
# def extract_grades(description, grade_list):
#     found_grades = []
#     for grade in grade_list:
#         pattern = rf"\b{re.escape(grade)}\b"  # Exact word match
#         matches = re.findall(pattern, str(description), re.IGNORECASE)
#         if matches:
#             found_grades.extend(matches)  # Store all matches

#     return ", ".join(set(found_grades)) if found_grades else "Other"  # Remove duplicates & join

# # Function to extract the first occurrence of finish
# def extract_finish(description, finish_list):
#     for finish in finish_list:
#         pattern = rf"\b{re.escape(finish)}\b"  # Exact word match
#         if re.search(pattern, str(description), re.IGNORECASE):
#             return finish
#     return "?"  # Add '?' if no finish is found

# # Function to extract quality
# def extract_quality(description):
#     match = re.search(r'\b(Prime|Stock|Second)\b', str(description), re.IGNORECASE)
#     return match.group(1) if match else '?'  # Returns '?' if no match is found

# # Ensure columns exist in DataFrame before updating
# df["Grade"] = df["PRODUCT DESCRIPTION"].apply(lambda x: extract_grades(x, grade_list))
# df["Finish"] = df["PRODUCT DESCRIPTION"].apply(lambda x: extract_finish(x, finish_list))
# df["Quality"] = df["PRODUCT DESCRIPTION"].apply(lambda x: extract_quality(x))

# # Save the updated DataFrame to a new Excel file
# output_file = "output_dem.xlsx"
# df.to_excel(output_file, index=False)

# print(f"Extraction completed! Results saved to {output_file}")

import pandas as pd
import json
import re

# Load Excel file
file_path = "modified_file.xlsx"  # Update with your actual file path
df = pd.read_excel(file_path)

# Load grade and finish lists from JSON
with open("Grades_json.json", "r") as f:
    grade_list = json.load(f)  # List of possible grades

with open("Finish_json.json", "r") as f:
    finish_list = json.load(f)  # List of possible finishes

# Function to extract the longest matching grade
def extract_grades(description, grade_list):
    found_grades = []
    for grade in grade_list:
        pattern = rf"\b{re.escape(grade)}\b"  # Exact word match
        if re.search(pattern, str(description), re.IGNORECASE):
            found_grades.append(grade)

    return max(found_grades, key=len) if found_grades else "Other"  # Select longest match

# Function to extract the first occurrence of finish
def extract_finish(description, finish_list):
    for finish in finish_list:
        pattern = rf"\b{re.escape(finish)}\b"  # Exact word match
        if re.search(pattern, str(description), re.IGNORECASE):
            return finish
    return "?"  # Add '?' if no finish is found

# Function to extract quality
def extract_quality(description):
    match = re.search(r'\b(Prime|Stock|Second)\b', str(description), re.IGNORECASE)
    return match.group(1) if match else '?'  # Returns '?' if no match is found

# Ensure columns exist in DataFrame before updating
df["Grade"] = df["PRODUCT DESCRIPTION"].apply(lambda x: extract_grades(x, grade_list))
df["Finish"] = df["PRODUCT DESCRIPTION"].apply(lambda x: extract_finish(x, finish_list))
df["Quality"] = df["PRODUCT DESCRIPTION"].apply(lambda x: extract_quality(x))

# Save the updated DataFrame to a new Excel file
output_file = "output_dem.xlsx"
df.to_excel(output_file, index=False)

print(f"Extraction completed! Results saved to {output_file}")


Extraction completed! Results saved to output_dem.xlsx


In [27]:
import pandas as pd
import re

def extract_width_thickness(description):
    description = str(description).upper().strip()  # Normalize case and handle NaN

    thickness = None
    width = None

    # Case with "T" notation
    primary_pattern = r'(\d+(\.\d+)?)\s*T\s*[*Xx]?\s*(\d+(\.\d+)?)\s*MM(?:[*Xx]?(\d+(\.\d+)?)\s*MM)?'
    match = re.search(primary_pattern, description)

    if match:
        thickness = float(match.group(1))  # Extract Thickness
        width = float(match.group(3)) if match.lastindex and match.lastindex >= 3 else None  # Extract Width
    else:
        # Pattern to capture ranges like "0.395-0.488 MM X 130-200 MM"
        range_pattern = r'(\d+(\.\d+)?)-(\d+(\.\d+)?)\s*MM\s*[Xx]\s*(\d+(\.\d+)?)-(\d+(\.\d+)?)\s*MM'
        range_match = re.search(range_pattern, description)

        if range_match:
            thickness = f"{range_match.group(1)}-{range_match.group(3)}"
            width = f"{range_match.group(5)}-{range_match.group(7)}"
        else:
            # Other common patterns
            patterns = [
                r'THICKNESS[-:\s]*(\d+(\.\d+)?)',
                r'WIDTH[:\s-]*(\d+(\.\d+)?)\s*MM',
                r'THICK(?:NESS)?[:\s-]*(\d+(\.\d+)?)\s*MM',
                r'THICKNESS[-\s]*(\d+(\.\d+)?)\s*MM\s*X\s*WIDTH[-\s]*(\d+(\.\d+)?)\s*MM',
                r'(\d+(\.\d+)?)\s*MM\s*[*Xx]?\s*(\d+(\.\d+)?)\s*MM',
                r'(\d+(\.\d+)?)\s*[*Xx]?\s*(\d+(\.\d+)?)\s*[*Xx]?\s*(\d+(\.\d+)?)\s*MM',
                r'SIZE.*?(\d+(\.\d+)?)\s*MM\s*[*Xx]?\s*(\d+(\.\d+)?)\s*MM',
                r'(\d+(\.\d+)?)\s*MM\s*X\s*(\d+(\.\d+)?)\s*MM',
            ]

            for pattern in patterns:
                match = re.search(pattern, description)
                if match:
                    if "THICKNESS" in pattern and "WIDTH" in pattern:
                        thickness = float(match.group(1))
                        width = float(match.group(3))
                    elif "THICKNESS" in pattern:
                        thickness = float(match.group(1))
                    elif "WIDTH" in pattern:
                        width = float(match.group(1))
                    else:
                        thickness = float(match.group(1))
                        width = float(match.group(3)) if match.lastindex and match.lastindex >= 3 else None
                    break  # Stop after first valid match

    # Format values for consistency
    if thickness and isinstance(thickness, float):
        thickness = f"{thickness:.3f}"
    if width and isinstance(width, float):
        width = f"{width:.2f}"

    return thickness, width

# Load Excel file
input_file = "output_dem.xlsx"
df = pd.read_excel(input_file)

# Apply extraction function to each row
df[['Thickness', 'Width']] = df['PRODUCT DESCRIPTION'].apply(lambda x: pd.Series(extract_width_thickness(x)))

# Save the updated file
output_file = "updated_" + input_file
df.to_excel(output_file, index=False)

print(f"Processing complete! Updated file saved as: {output_file}")


Processing complete! Updated file saved as: updated_output_dem.xlsx
